In [64]:
import numpy as np
import pandas as pd
import quandl as q
from matplotlib import pyplot as plt
import seaborn as sb
import math 
from IPython.display import display
pd.options.display.max_columns = None
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [41]:
q.ApiConfig.api_key = 
q.ApiConfig.api_version = '2017-11-02'

In [42]:
df = q.get('WIKI/AAPL',start_date = '2017-01-01')
df['30MA'] = df['Adj. Close'].rolling(30).mean()
df['60MA'] = df['Adj. Close'].rolling(60).mean()

In [43]:
df['Long'] = np.where(df['30MA'] > df['60MA'],df['Adj. High'],np.NaN)
df['Short'] = np.where(df['30MA'] < df['60MA'],df['Adj. Low'],np.NaN)

In [45]:
condition = [np.isnan(df['Long']), (df['Long'] == df['Adj. High']) & (np.isnan(df['Long'].shift()))]
choices = [np.NaN, df['Adj. High']]
shortcondition = [np.isnan(df['Short']), (df['Short'] == df['Adj. Low']) & (np.isnan(df['Short'].shift()))]
shortchoices = [np.NaN, df['Adj. Low']]

In [50]:
df['LongBuyLimit'] = np.select(condition,choices,default=np.nan)
df['ShortBuyLimit'] = np.select(shortcondition,shortchoices,default=np.nan)

In [53]:
df['BuyLimitFill'] = np.where(df['Long'] > 1, df['LongBuyLimit'].ffill(), np.NaN)
df['ShortLimitFill'] = np.where(df['Short'] > 1, df['ShortBuyLimit'].ffill(), np.NaN)

In [10]:
df['entry'] = np.where((df['30MA'] >= df['60MA']) & (df['Adj. High'] > df['CrossOver2']), 1 , 0)



In [66]:
d = df[['Adj. High','Adj. Low','30MA', '60MA', 'Long','Short','LongBuyLimit','ShortBuyLimit','BuyLimitFill','ShortLimitFill']].tail(500)

In [67]:
display(d)

,Adj. High,Adj. Low,30MA,60MA,Long,Short,LongBuyLimit,ShortBuyLimit,BuyLimitFill,ShortLimitFill
Date,,,,,,,,,,
2017-01-03,114.893155,113.342546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,115.070931,114.320318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,115.420756,114.379577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-06,116.700551,115.031425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-09,117.954865,116.483269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-10,117.905483,116.838822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-11,118.448689,117.135117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-12,117.826471,116.749934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-13,118.142518,117.342523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.loc[df['entry'] == 1]

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,30MA,60MA,CrossOver,CrossOver2,entry
Date,,,,,,,,,,,,,,,,,
2017-03-30,144.19,143.329529,143.5000,143.9300,21207252.0,0.00,1.0,143.022041,143.329529,142.337630,142.764147,21207252.0,138.288529,130.488876,143.329529,143.31961,1
2017-04-04,143.25,143.716370,143.1700,144.7700,19891354.0,0.00,1.0,142.089655,143.716370,142.010303,143.597342,19891354.0,139.094117,131.862532,143.716370,143.31961,1
2017-04-05,144.22,144.281753,143.8100,144.0200,27717854.0,0.00,1.0,143.051798,144.281753,142.645119,142.853418,27717854.0,139.322585,132.284750,144.281753,143.31961,1
2017-04-06,144.29,143.349367,143.4500,143.6600,21149034.0,0.00,1.0,143.121231,143.349367,142.288035,142.496334,21149034.0,139.558326,132.699042,143.349367,143.31961,1
2017-04-25,143.91,143.726289,143.8700,144.5400,18216472.0,0.00,1.0,142.744309,143.726289,142.704633,143.369205,18216472.0,140.999887,137.165029,143.726289,143.31961,1
2017-04-26,144.47,143.428719,143.3762,143.6508,19614287.0,0.00,1.0,143.299772,143.428719,142.214832,142.487208,19614287.0,141.153989,137.537688,143.428719,143.31961,1
2017-05-01,145.10,146.007659,144.9600,146.6000,32818760.0,0.00,1.0,143.924669,146.007659,143.785803,145.412519,32818760.0,141.580506,138.480535,146.007659,143.31961,1
2017-05-02,147.54,146.890450,146.8400,147.5100,39752670.0,0.00,1.0,146.344905,146.890450,145.650575,146.315148,39752670.0,141.780539,138.794360,146.890450,143.31961,1
2017-05-03,145.59,146.295310,144.2700,147.0600,45142806.0,0.00,1.0,144.410700,146.295310,143.101393,145.868793,45142806.0,142.019256,139.080828,146.295310,143.31961,1


In [87]:
signals = pd.DataFrame(index=df.index)
signals['signals'] = 0.0
signals['short_mavg'] = df['Adj. Close'].rolling(window=40,min_periods=1,center=False).mean()
signals['long_mavg'] = df['Adj. Close'].rolling(window=100,min_periods=1,center=False).mean()

In [177]:
print(signals)

            signals  short_mavg   long_mavg
Date                                       
2017-01-03      0.0  114.715378  114.715378
2017-01-04      0.0  114.651181  114.651181
2017-01-05      0.0  114.824019  114.824019
2017-01-06      0.0  115.231424  115.231424
2017-01-09      0.0  115.689199  115.689199
2017-01-10      0.0  116.014136  116.014136
2017-01-11      0.0  116.336532  116.336532
2017-01-12      0.0  116.516602  116.516602
2017-01-13      0.0  116.633611  116.633611
2017-01-17      0.0  116.822032  116.822032
2017-01-18      0.0  116.975297  116.975297
2017-01-19      0.0  117.085734  117.085734
2017-01-20      0.0  117.195895  117.195895
2017-01-23      0.0  117.295962  117.295962
2017-01-24      0.0  117.375445  117.375445
2017-01-25      0.0  117.562892  117.562892
2017-01-26      0.0  117.731773  117.731773
2017-01-27      0.0  117.882437  117.882437
2017-01-30      0.0  118.000609  118.000609
2017-01-31      0.0  118.093136  118.093136
2017-02-01      0.0  118.524879 

In [178]:
df['return'] = ((df['Adj. Close'])/df['Adj. Close'].shift(1)-1)*100
df['return2'] = np.log(df['Adj. Close'].pct_change()+1)*100

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,return,return2,rolling50
Date,,,,,,,,,,,,,,,
2017-01-03,115.80,116.3300,114.76,116.15,28781865.0,0.0,1.0,114.369701,114.893155,113.342546,114.715378,28781865.0,NaN,NaN,NaN
2017-01-04,115.85,116.5100,115.75,116.02,21118116.0,0.0,1.0,114.419083,115.070931,114.320318,114.586983,21118116.0,-0.111924,-0.111987,NaN
2017-01-05,115.92,116.8642,115.81,116.61,22193587.0,0.0,1.0,114.488219,115.420756,114.379577,115.169696,22193587.0,0.508533,0.507244,NaN
2017-01-06,116.78,118.1600,116.47,117.91,31751900.0,0.0,1.0,115.337596,116.700551,115.031425,116.453639,31751900.0,1.114827,1.108659,NaN
2017-01-09,117.95,119.4300,117.94,118.99,33561948.0,0.0,1.0,116.493145,117.954865,116.483269,117.520300,33561948.0,0.915953,0.911783,NaN


In [179]:
df['dailyret'] = df['return']-.04/252

In [180]:
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,return,return2,rolling50,dailyret
Date,,,,,,,,,,,,,,,,
2017-01-03,115.80,116.3300,114.76,116.15,28781865.0,0.0,1.0,114.369701,114.893155,113.342546,114.715378,28781865.0,NaN,NaN,NaN,NaN
2017-01-04,115.85,116.5100,115.75,116.02,21118116.0,0.0,1.0,114.419083,115.070931,114.320318,114.586983,21118116.0,-0.111924,-0.111987,NaN,-0.112083
2017-01-05,115.92,116.8642,115.81,116.61,22193587.0,0.0,1.0,114.488219,115.420756,114.379577,115.169696,22193587.0,0.508533,0.507244,NaN,0.508374
2017-01-06,116.78,118.1600,116.47,117.91,31751900.0,0.0,1.0,115.337596,116.700551,115.031425,116.453639,31751900.0,1.114827,1.108659,NaN,1.114668
2017-01-09,117.95,119.4300,117.94,118.99,33561948.0,0.0,1.0,116.493145,117.954865,116.483269,117.520300,33561948.0,0.915953,0.911783,NaN,0.915794


In [181]:
sharp = math.sqrt(252)*df['dailyret'].mean()/df['dailyret'].std()
print(sharp)

2.55449858207


In [182]:
del df['Volume']

In [184]:
#df['cummaltive'] = (df['dailyret'].shift(1)+1)*(df['dailyret']+1)-1
df['cummaltive'] = (df['dailyret']+1).cumprod()-1

In [185]:
df.plot(x=df.index.values,y='Adj. Close')

,Open,High,Low,Close,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,return,return2,rolling50,dailyret,cummaltive
Date,,,,,,,,,,,,,,,,
2017-01-03,115.80,116.3300,114.76,116.15,0.0,1.0,114.369701,114.893155,113.342546,114.715378,28781865.0,NaN,NaN,NaN,NaN,NaN
2017-01-04,115.85,116.5100,115.75,116.02,0.0,1.0,114.419083,115.070931,114.320318,114.586983,21118116.0,-0.111924,-0.111987,NaN,-0.112083,-0.112083
2017-01-05,115.92,116.8642,115.81,116.61,0.0,1.0,114.488219,115.420756,114.379577,115.169696,22193587.0,0.508533,0.507244,NaN,0.508374,0.339311
2017-01-06,116.78,118.1600,116.47,117.91,0.0,1.0,115.337596,116.700551,115.031425,116.453639,31751900.0,1.114827,1.108659,NaN,1.114668,1.832199
2017-01-09,117.95,119.4300,117.94,118.99,0.0,1.0,116.493145,117.954865,116.483269,117.520300,33561948.0,0.915953,0.911783,NaN,0.915794,4.425911
